## Human in the Loop

<img src="./assets/human-in-the-loop.png" width="500">

In [20]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [21]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db_ctx: SQLDatabase

In [22]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQL query and return the results as a string."""
    runtime = get_runtime(RuntimeContext)
    db_tool = runtime.context.db_ctx
    try:
        return db_tool.run(query)
    except Exception as e:
        return f"Error: {e}"

In [23]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step by step.
- When you need data, call the tool `execute_sql` with one SELECT query.
- Read-only only: no INSERT, UPDATE, DELETE, ALTER, DROP, CREATE, REPLACE, TRUNCATE statements.
- Limit to 5 rows unless the user specifies otherwise.
- If the tool returns an error, fix the query and try again.
- Prefer explicit column lists instead of SELECT *.
- If the database is offline, ask user to try again later without further comment
"""

In [24]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="ollama:gpt-oss:20b-cloud",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=InMemorySaver(),
    context_schema=RuntimeContext,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"execute_sql": {"allowed_decisions":["approve", "reject"]}},
        )
    ],
)

In [25]:
from langgraph.types import Command

question = "What are the names of all the employees?"

config = {"configurable": {"thread_id": "1"}}

result = agent.invoke(
    {"messages": {"role": "user", "content": question}},
    config=config,
    context=RuntimeContext(db_ctx=db),
)

# print(result["__interrupt__"][-1].value["action_requests"][-1]["description"])

if "__interrupt__" in result:
    description = result["__interrupt__"][-1].value["action_requests"][-1]["description"]
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(f"\033[1;3;31m Interrupt: {description}\033[0m]")
    
    result = agent.invoke(
        Command(
            resume={
                "decisions": [{"type": "reject", "message": "the database is offline."}]
            }
        ),
        config=config,
        context=RuntimeContext(db_ctx=db),
    )
    print(f"\033[1;3;31m{80 * '-'}\033[0m")

print(result["messages"][-1].content)

--------------------------------------------------------------------------------
 Interrupt: Tool execution requires approval

Tool: execute_sql
Args: {'query': "SELECT name FROM sqlite_master WHERE type='table';"}]
--------------------------------------------------------------------------------
I’m sorry, but the database isn’t available right now. Please try your request again later.


In [28]:
from langgraph.types import Command

question = "What are the names of all the employees?"

config = {"configurable": {"thread_id": "2"}}

result = agent.invoke(
    {"messages": {"role": "user", "content": question}},
    config=config,
    context=RuntimeContext(db_ctx=db),
)

if "__interrupt__" in result:
    description = result["__interrupt__"][-1].value["action_requests"][-1]["description"]
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(f"\033[1;3;31m Interrupt: {description}\033[0m]")
    
    result = agent.invoke(
        Command(
            resume={
                "decisions": [{"type": "approve"}]
            }
        ),
        config=config,
        context=RuntimeContext(db_ctx=db),
    )
    print(f"\033[1;3;31m{80 * '-'}\033[0m")

for msg in result["messages"]:
    msg.pretty_print()

--------------------------------------------------------------------------------
 Interrupt: Tool execution requires approval

Tool: execute_sql
Args: {'query': 'SELECT FirstName, LastName FROM Employee;'}]
--------------------------------------------------------------------------------
================================ Human Message =================================

What are the names of all the employees?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (c9e09dfb-3972-4ced-a86c-e7651da92dc9)
 Call ID: c9e09dfb-3972-4ced-a86c-e7651da92dc9
  Args:
    query: SELECT name FROM employees LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: employees
[SQL: SELECT name FROM employees LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ================================